## AUTOMATIZACIÓN CARGA Y PROCESADO DE TABLAS FORMATO 'EHIS' O SIMILAR

### Librerías necesarias

In [1]:
import numpy as np
import pandas as pd
import os

### Función "carga_formato_independientes_EHIS()".

Función que a partir de los archivos 'csv' descargados de la página de Eurostat con el formato EHIS o similar crea un archivo por cada variable ya con el formato adecuado (pivotaje), siendo sus columnas las de las dimensiones geográfica (GEO), de género (SEX) y grupos de edad (AGE), y por último una columna por cada categoría de la variable en cuestión con sus valores correspondientes.

Tiene como entrada un diccionario de tuplas (nombre_archivo_descarga_variable, nombre_columna_categorias, categorías_por_orden)


In [2]:
def carga_formato_independientes_EHIS(dict_archivos):
    
    PAISES_ESTUDIO = {'Austria','Belgium','Bulgaria','Croatia','Cyprus','Czechia','Denmark','Estonia','Finland','France', \
                     'Germany (until 1990 former territory of the FRG)','Greece','Hungary','Iceland','Ireland','Italy', \
                     'Latvia','Lithuania','Luxembourg','Malta','Netherlands','Norway','Poland','Portugal','Romania', \
                     'Slovakia','Slovenia','Spain','Sweden','Turkey','United Kingdom'}

    # inicialización variables necesarias
    lista_fallos = []
    n_var_indep = 0
    total_var_indep = len(dict_archivos)
    
    
    # carga del archivo original (Eurostat), procesado, comprobación y guardado en su caso
    # del archivo correspondiente a cada variable
    
    for var_indep,archivo in dict_archivos.items():
        
        #inicialización y carga archivo
        n_var_indep +=1
        ruta = os.path.join('.\data\origen',archivo[0])
        print('#'*30)
        print('CARGANDO ARCHIVO {} EN DF ({} de {}) ...'.format(ruta,n_var_indep,total_var_indep))
        print('#'*30,'\n'*2)
        
        df_varind = pd.read_csv(ruta, na_values = [': u',':'])
        
        print(df_varind.info(),'\n'*2)
        print(df_varind.head(),'\n'*2)
        
        # Eliminar columnas no necesarias
        print('Eliminamos columnas innecesarias ...')
        df_varind.drop(columns= ['UNIT', 'TIME','ISCED11','Flag and Footnotes'], inplace=True)
        print('Columnas que quedan: ', df_varind.columns.values,'\n')
        
        # reordenamos columnas para formato común
        print('Reordenamos columnas ...')
        df_varind = df_varind[['GEO','SEX','AGE',archivo[1],'Value']]
        print('Columnas quedan: ', df_varind.columns.values,'\n')
        
        # comprobación dimensiones correctas
        print('Comprobando longitud dimensiones ...','\n'*2)
        unicos_GEO,unicos_SEX,unicos_AGE = len(df_varind.GEO.unique()),len(df_varind.SEX.unique()),len(df_varind.AGE.unique())
        print('GEO nº valores unicos: {}'.format(unicos_GEO),'\n')
        print('DIFERENCIA: ', PAISES_ESTUDIO-set(df_varind.GEO.unique()),'\n')
        print('SEX nº valores unicos: {}'.format(unicos_SEX),'\n',df_varind.SEX.unique(),'\n')
        print('AGE nº valores unicos: {}'.format(unicos_AGE),'\n',df_varind.AGE.unique(),'\n'*2)
        salto = unicos_GEO*unicos_SEX*unicos_AGE
                
        # Pivotaje: columna de la variable (pasada como argumento) y columna 'Value'
        print('Empezando proceso de formato, aplicando pivot ...','\n'*2)
        dftemp = df_varind.copy()
        dftemp.reset_index(inplace=True)
        dftemp_ancho = dftemp.pivot(index='index',columns=archivo[1],values='Value')
        print('Columnas temporal ancho son: ', dftemp_ancho.columns.values,'\n'*2)
        print('DF del pivote:','\n'*2)
        print(dftemp_ancho,'\n'*2)
        
        print('Imponemos mismo orden categorias de la independiente que en el archivo original','\n', \
              '(metodo "pivot" los reordena por alfabetico) ...','\n'*2)
        dftemp_ancho = dftemp_ancho[archivo[2]]
        print('DF del pivote reordenado:','\n'*2)
        
        # comprobación correspondencia archivo original
        print(dftemp_ancho, '\n'*2,'Comprobamos correspondencia con el original....','\n')
        dfmuestratemp_ancho = dftemp_ancho.sample(n=10)
        dfmuestratemp_ancho['correspondencia'] = df_varind.loc[dfmuestratemp_ancho.index,archivo[1]]
        print(dfmuestratemp_ancho,'\n'*2)
        
        
        # CREACIÓN DATASET DE SALIDA
        
        # columnas dimensiones
        print('Creando DF de salida de la variable (columnas con categorias var indep) ...','\n'*2)
        df_salida = df_varind.loc[0:salto-1,['GEO','SEX','AGE']]
        
        lista_cat_indep = dftemp_ancho.columns.values
        lista_prefijo_varindep = [var_indep+'_']*len(lista_cat_indep)
        
        # formato nombres columnas
        columnas_categ_indep =[]
        for prefijo,sufijo in zip(lista_prefijo_varindep,lista_cat_indep):
            columnas_categ_indep.append(prefijo+sufijo)
        #DEBUG
        #print('DEBUG:\n','Columnas categoricas independientes:')
        #print(columnas_categ_indep,'\n'*2)
              
        # inicialización comprobación
        inicio = 0
        columna_suma_porc = pd.Series(data=0,index=df_salida.index)        
        #DEBUG
        #print('DEBUG:\n','inicializacion columna suma porc:')
        #print('Valores unicos:',columna_suma_porc.unique(),'Longitud:',len(columna_suma_porc),'\n'*2)
        
        # una columna por cada categoría, adecuación a partir del proceso 'pivotaje'
        for n,columna in enumerate(columnas_categ_indep):            
            df_salida[columna] = dftemp_ancho.loc[inicio:inicio+salto-1,lista_cat_indep[n]].values
            inicio = inicio + salto
            columna_suma_porc = columna_suma_porc + df_salida[columna]
            
        print(df_salida.info(),'\n'*2, df_salida.head(),'\n'*2)
        
        
        # salidas comprobaciones
        
        print('Comprobacion: suma de porcentajes, valores unicos...','\n',columna_suma_porc.unique(),'\n'*2)
        
        print('Ultima comprobacion: cotejo con el original...','\n'*2)
        
        print('Muestra DF procesado...','\n'*2)
        df_cotejo = df_salida.sample(n=5)
        print(df_cotejo,'\n'*2)
        
        print('Buscando valores correspondientes en el archivo ORIGINAL:','\n'*2)
        for registro in df_cotejo.itertuples():
            print(df_varind[(df_varind.GEO==registro.GEO) & (df_varind.SEX==registro.SEX) & (df_varind.AGE==registro.AGE)])
            print('\n')
                         
        
        # opción de guardar o no según comprobación
        if str.upper(input('Pausa comprobación, pulsa "S" si correcto:\t')) == 'S':
            
            # guardado archivo (dimensiones + 1 columna por categoria con sus valores)
            
            print('{} procesado con exito ...'.format(var_indep))
            nombre_guardar = 'base_' + var_indep + '.csv'
            guardar_ruta = os.path.join('.\data\procesado', nombre_guardar)
            df_salida.to_csv(guardar_ruta, index=False)
            print('{} guardado como {}.'.format(var_indep,guardar_ruta),'\n'*2)
            
        else:
            
            # salida con los errores
            
            lista_fallos.append(var_indep)
            print('{} no es correcto.'.format(var_indep))
            
    
    # salida final
    print('Procesado no dio resultado en {} de {}'.format(len(lista_fallos),total_var_indep))
    print(lista_fallos)
        
            

### Llamada a la función

In [3]:
variables = {'DailyFyV': ('hlth_ehis_fv3e_1_Data.csv','N_PORTION',["0 portions","From 1 to 4 portions","5 portions or more"]), \
             'smoking': ('hlth_ehis_sk1e_1_Data.csv',"SMOKING",["Non-smoker","Daily smoker", "Occasional smoker"]), \
            'Freq_alcohol': ('hlth_ehis_al1e_1_Data.csv',"FREQUENC",["Every day","Every week","Every month", \
                                                                "Less than once a month","Never or not in the last 12 months"]), \
            'social_support': ('hlth_ehis_ss1e_1_Data.csv','LEV_PERC',["Strong","Intermediate","Poor"]), \
            'BMI': ('hlth_ehis_bm1e_1_Data.csv',"BMI",["Underweight","Normal","Overweight"]), \
            'Weekly_phys_actv': ('hlth_ehis_pe2e_1_Data.csv',"DURATION",["Zero minutes","From 1 to 149 minutes", \
                                                                        "From 150 to 299 minutes","300 minutes or over"])}


In [4]:
carga_formato_independientes_EHIS(variables)

##############################
CARGANDO ARCHIVO .\data\origen\hlth_ehis_fv3e_1_Data.csv EN DF (1 de 6) ...
############################## 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 9 columns):
N_PORTION             930 non-null object
GEO                   930 non-null object
UNIT                  930 non-null object
TIME                  930 non-null int64
ISCED11               930 non-null object
SEX                   930 non-null object
AGE                   930 non-null object
Value                 930 non-null float64
Flag and Footnotes    0 non-null float64
dtypes: float64(2), int64(1), object(6)
memory usage: 65.5+ KB
None 


    N_PORTION      GEO        UNIT  TIME                 ISCED11    SEX  \
0  0 portions  Belgium  Percentage  2014  All ISCED 2011 levels   Males   
1  0 portions  Belgium  Percentage  2014  All ISCED 2011 levels   Males   
2  0 portions  Belgium  Percentage  2014  All ISCED 2011 levels   Males   
3  0 po

Pausa comprobación, pulsa "S" si correcto:	s
DailyFyV procesado con exito ...
DailyFyV guardado como .\data\procesado\base_DailyFyV.csv. 


##############################
CARGANDO ARCHIVO .\data\origen\hlth_ehis_sk1e_1_Data.csv EN DF (2 de 6) ...
############################## 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 9 columns):
SMOKING               930 non-null object
GEO                   930 non-null object
UNIT                  930 non-null object
ISCED11               930 non-null object
SEX                   930 non-null object
AGE                   930 non-null object
TIME                  930 non-null int64
Value                 930 non-null float64
Flag and Footnotes    30 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 65.5+ KB
None 


      SMOKING      GEO        UNIT                 ISCED11    SEX  \
0  Non-smoker  Belgium  Percentage  All ISCED 2011 levels   Males   
1  Non-smoker  Belgium  Perce

Pausa comprobación, pulsa "S" si correcto:	s
smoking procesado con exito ...
smoking guardado como .\data\procesado\base_smoking.csv. 


##############################
CARGANDO ARCHIVO .\data\origen\hlth_ehis_al1e_1_Data.csv EN DF (3 de 6) ...
############################## 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 9 columns):
FREQUENC              1450 non-null object
GEO                   1450 non-null object
UNIT                  1450 non-null object
TIME                  1450 non-null int64
ISCED11               1450 non-null object
SEX                   1450 non-null object
AGE                   1450 non-null object
Value                 1450 non-null float64
Flag and Footnotes    85 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 102.1+ KB
None 


    FREQUENC      GEO        UNIT  TIME                 ISCED11    SEX  \
0  Every day  Belgium  Percentage  2014  All ISCED 2011 levels   Males   
1  Every da

Pausa comprobación, pulsa "S" si correcto:	s
Freq_alcohol procesado con exito ...
Freq_alcohol guardado como .\data\procesado\base_Freq_alcohol.csv. 


##############################
CARGANDO ARCHIVO .\data\origen\hlth_ehis_ss1e_1_Data.csv EN DF (4 de 6) ...
############################## 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 9 columns):
LEV_PERC              930 non-null object
GEO                   930 non-null object
UNIT                  930 non-null object
TIME                  930 non-null int64
ISCED11               930 non-null object
SEX                   930 non-null object
AGE                   930 non-null object
Value                 930 non-null float64
Flag and Footnotes    27 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 65.5+ KB
None 


  LEV_PERC      GEO        UNIT  TIME                 ISCED11    SEX  \
0   Strong  Belgium  Percentage  2014  All ISCED 2011 levels   Males   
1   Strong 

Pausa comprobación, pulsa "S" si correcto:	s
BMI procesado con exito ...
BMI guardado como .\data\procesado\base_BMI.csv. 


##############################
CARGANDO ARCHIVO .\data\origen\hlth_ehis_pe2e_1_Data.csv EN DF (6 de 6) ...
############################## 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 9 columns):
DURATION              1200 non-null object
GEO                   1200 non-null object
UNIT                  1200 non-null object
TIME                  1200 non-null int64
ISCED11               1200 non-null object
SEX                   1200 non-null object
AGE                   1200 non-null object
Value                 1192 non-null float64
Flag and Footnotes    52 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 84.5+ KB
None 


       DURATION      GEO        UNIT  TIME                 ISCED11    SEX  \
0  Zero minutes  Belgium  Percentage  2014  All ISCED 2011 levels   Males   
1  Zero minutes  B

Pausa comprobación, pulsa "S" si correcto:	s
Weekly_phys_actv procesado con exito ...
Weekly_phys_actv guardado como .\data\procesado\base_Weekly_phys_actv.csv. 


Procesado no dio resultado en 0 de 6
[]


**Comprobamos uno de los archivos:**

In [7]:
df_prueba = pd.read_csv(os.path.join('.\data\procesado','base_BMI.csv'))
df_prueba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 6 columns):
GEO                310 non-null object
SEX                310 non-null object
AGE                310 non-null object
BMI_Underweight    310 non-null float64
BMI_Normal         310 non-null float64
BMI_Overweight     310 non-null float64
dtypes: float64(3), object(3)
memory usage: 14.7+ KB


nº observaciones y columnas correcto

In [8]:
df_prueba.head()

,GEO,SEX,AGE,BMI_Underweight,BMI_Normal,BMI_Overweight
0,Belgium,Males,From 15 to 24 years,11.9,69.4,18.7
1,Belgium,Males,From 25 to 34 years,2.4,56.5,41.0
2,Belgium,Males,From 35 to 44 years,0.3,45.6,54.1
3,Belgium,Males,From 45 to 64 years,0.4,33.8,65.8
4,Belgium,Males,65 years or over,0.6,33.7,65.6


In [9]:
df_prueba.tail()

,GEO,SEX,AGE,BMI_Underweight,BMI_Normal,BMI_Overweight
305,Turkey,Females,From 15 to 24 years,17.8,64.8,17.4
306,Turkey,Females,From 25 to 34 years,5.1,56.3,38.7
307,Turkey,Females,From 35 to 44 years,1.9,34.4,63.8
308,Turkey,Females,From 45 to 64 years,0.7,18.9,80.4
309,Turkey,Females,65 years or over,2.3,30.3,67.4
